In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

from nimosef.utils.stats import pairwise_comparisons

In [ ]:
data_path="/media/jaume/DATA/Data/Test_NIMOSEF_Dataset"
splits_filename=f"{data_path}/derivatives/manifests_nimosef/dataset_manifest.json"

mode="train"
results_folder = f"{data_path}/derivatives/nimosef_results"
save_folder_results = os.path.join(results_folder, f"results_{mode}_comparison")
os.makedirs(save_folder_results, exist_ok=True)

# Metadata filename
metadata_filename = os.path.join(data_path, "derivatives", "metadata_participants_ALL.tsv")
df_metadata = pd.read_csv(metadata_filename, sep='\t')
df_metadata['Subject'] = df_metadata['Subject'].astype(str)
df_metadata['bsa_img_2.0'] = df_metadata['bsa_img_2.0'].astype(np.float32)

In [ ]:
# Image metrics
volumes_df_path = os.path.join(save_folder_results, 'volumes_imgs.parquet')
df_volumes = pd.read_parquet(volumes_df_path)
df_volumes["Subject"] = df_volumes["Subject"].astype(str)

final_df_path = os.path.join(save_folder_results, 'connected_components.parquet')
final_df = pd.read_parquet(final_df_path)

dice_df_path = os.path.join(save_folder_results, 'dice_score.parquet')
df_dice_final = pd.read_parquet(dice_df_path)

label_map = {1.0: 'LV', 2.0: 'MYO', 3.0: 'RV'}
final_df['label'] = final_df['label'].map(label_map)

In [ ]:
# Merge df_all with df_metadata by using 'Subject' from df_all and the index of df_metadata
df_all_merged = pd.merge(df_volumes, df_metadata[['bsa_img_2.0', 'Subject']], left_on="Subject", right_on="Subject", how="left")

# Optionally, rename the column for clarity
df_all_merged.rename(columns={'bsa_img_2.0': 'BSA'}, inplace=True)

# Compute the Volume_Index column as Volume divided by BSA
df_all_merged['Volume_Index'] = df_all_merged['Volume'] / df_all_merged['BSA']

# Display the first few rows to check the merge
print(df_all_merged.head())

In [ ]:
# ---------------------------
# Compute group-level statistics: mean and 95% confidence intervals (CI)
# ---------------------------
# Group the data by segmentation type, time, and class
grouped = df_all_merged.groupby(['segmentation_type', 'Time', 'Class'])

# Aggregate the data: mean, std, and count
summary = grouped.agg(
    mean_volume=('Volume', 'mean'),
    std_volume=('Volume', 'std'),
    count_volume=('Volume', 'count'),
    mean_deriv=('Derivative', 'mean'),
    std_deriv=('Derivative', 'std'),
    count_deriv=('Derivative', 'count'),
    mean_volume_index=('Volume_Index', 'mean'),
    std_volume_index=('Volume_Index', 'std'),
    count_volume_index=('Volume_Index', 'count'),
).reset_index()

# Compute the standard error and 95% CI (assuming normality, 1.96 * SE)
summary['se_volume'] = summary['std_volume'] / np.sqrt(summary['count_volume'])
summary['ci_volume'] = summary['se_volume'] * 1.96

summary['se_deriv'] = summary['std_deriv'] / np.sqrt(summary['count_deriv'])
summary['ci_deriv'] = summary['se_deriv'] * 1.96

summary['se_volume_index'] = summary['std_volume_index'] / np.sqrt(summary['count_volume_index'])
summary['ci_volume_index'] = summary['se_volume_index'] * 1.96

# print(summary.head())
classes = df_all_merged['Class'].unique()
seg_types = df_all_merged['segmentation_type'].unique()

# title_dict = {'gt':'CNN reference', 'pred_new': 'Ours', 'pred_base': 'Baseline'}
title_dict = {'gt':'CNN reference', 'pred': 'Ours'}

In [ ]:
# Assuming 'summary' is the aggregated DataFrame from the previous code snippet
# and 'classes' is defined (e.g., ['LV', 'Myo', 'RV']).
# variable_to_plot = 'mean_volume'
# ci_to_plot = 'ci_volume'
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 24})
sns.set(style="whitegrid")
sns.set_context("talk", font_scale=1., 
                  rc={"font.size": 18, "axes.titlesize": 18, "axes.labelsize": 18,
                      "xtick.labelsize": 18, "ytick.labelsize": 18})

variable_to_plot = 'mean_volume_index'
ci_to_plot = 'ci_volume_index'

# Set up the figure with 3 horizontal subplots sharing the y-axis.
fig, axs = plt.subplots(ncols=2, figsize=(18, 6), sharey=True)

# Loop through each segmentation type and plot the data
for ax, seg in zip(axs, seg_types):
    df_seg = summary[summary['segmentation_type'] == seg]
    for cl in classes:
        df_cl = df_seg[df_seg['Class'] == cl]

        ax.plot(df_cl['Time'], df_cl[f'{variable_to_plot}'], label=f'{cl} Mean')
        ax.fill_between(df_cl['Time'],
                     df_cl[f'{variable_to_plot}'] - df_cl[f'{ci_to_plot}'],
                     df_cl[f'{variable_to_plot}'] + df_cl[f'{ci_to_plot}'],
                     alpha=0.3, label=f'{cl} 95% CI')
    
    ax.set_title(f'{title_dict[seg]}')
    ax.set_xlabel('Time')
    ax.grid(True)

# Set the common y-axis label on the left subplot
axs[0].set_ylabel('Volume Index [ml/m2]')

# Create one common legend for all subplots
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.0), ncol=len(classes)*2, frameon=True)

# Adjust layout to create space for the legend above the plots
fig.tight_layout(rect=[0, 0, 1, 0.95])

plt.show()

save_filename = os.path.join(save_folder_results, 'volume_index.png')
fig.savefig(save_filename, dpi=300, bbox_inches='tight', pad_inches=0)

save_filename = os.path.join(save_folder_results, 'volume_index.eps')
fig.savefig(save_filename, format='eps', dpi=300, bbox_inches='tight', pad_inches=0)

save_filename = os.path.join(save_folder_results, 'volume_index.pdf')
fig.savefig(save_filename, format='pdf', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# Set up the figure with 3 horizontal subplots sharing the y-axis.
fig, axs = plt.subplots(ncols=2, figsize=(18, 6), sharey=True)
# sns.set(style="whitegrid")
sns.set_context("talk", font_scale=1., 
                  rc={"font.size": 18, "axes.titlesize": 18, "axes.labelsize": 18,
                      "xtick.labelsize": 18, "ytick.labelsize": 18})

# Loop through each segmentation type and plot the derivative data
for ax, seg in zip(axs, seg_types):
    df_seg = summary[summary['segmentation_type'] == seg]
    for cl in classes:
        df_cl = df_seg[df_seg['Class'] == cl]

        ax.plot(df_cl['Time'], df_cl['mean_deriv'], marker='o', label=cl)
        ax.fill_between(
            df_cl['Time'],
            df_cl['mean_deriv'] - df_cl['ci_deriv'],
            df_cl['mean_deriv'] + df_cl['ci_deriv'],
            alpha=0.3
        )
    ax.set_title(f'{title_dict[seg]}')
    ax.set_xlabel('Time')
    ax.grid(True)

# Set a common y-axis label on the left subplot
axs[0].set_ylabel('Derivative (Change in Volume)')

# Create one common legend for all subplots
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.0), ncol=len(classes)*2, frameon=True)

# Adjust layout to create space for the legend above the plots
fig.tight_layout(rect=[0, 0, 1, 0.95])

plt.show()

save_filename = os.path.join(save_folder_results, 'volume_index_derivative.png')
fig.savefig(save_filename, dpi=300, bbox_inches='tight', pad_inches=0)

save_filename = os.path.join(save_folder_results, 'volume_index_derivative.eps')
fig.savefig(save_filename, format='eps', dpi=300, bbox_inches='tight', pad_inches=0)

save_filename = os.path.join(save_folder_results, 'volume_index_derivative.pdf')
fig.savefig(save_filename, format='pdf', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
# --- STEP 1: Compute per‐subject measures for each subject, segmentation, and class ---
# title_dict = {'gt':'CNN reference', 'pred_new': 'Ours', 'pred_base': 'Baseline'}
# We choose the variable to analyze (for volume, use 'Volume_Index'; otherwise, use 'Volume')
measure = 'Volume_Index'

# Compute minimum and maximum volume per subject
df_min_vol = df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])[measure].min().reset_index().rename(columns={measure: 'MinVolume'})
df_max_vol = df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])[measure].max().reset_index().rename(columns={measure: 'MaxVolume'})

# Compute EF = (Max - Min) / Max for each subject (merge min and max data)
df_ef = pd.merge(df_min_vol, df_max_vol, on=['Subject', 'segmentation_type', 'Class'])
df_ef['EF'] = (df_ef['MaxVolume'] - df_ef['MinVolume']) / df_ef['MaxVolume']
df_ef['EF'] = df_ef['EF'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Compute Stroke Volume (SV) as (MaxVolume - MinVolume)
# Merge the min and max volume dataframes
df_sv = pd.merge(df_min_vol, df_max_vol, on=['Subject', 'segmentation_type', 'Class'])
df_sv['SV'] = df_sv['MaxVolume'] - df_sv['MinVolume']

# For Derivative measures, assume df_all_merged has a 'Derivative' column.
# Compute per-subject min and max derivative for each subject, segmentation, and class.
df_deriv = (df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])['Derivative']
                .apply(lambda x: np.abs(x).max())
                .reset_index(name='MaxDeriv'))
# df_max_deriv = df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])['Derivative'].max().reset_index().rename(columns={'Derivative': 'MaxDeriv'})
# df_deriv = pd.merge(df_min_deriv, df_max_deriv, on=['Subject', 'segmentation_type', 'Class'])

# --- STEP 2: Create Boxplots for each Measure ---

# We will create boxplots for:
# - Minimum Volume, Maximum Volume, Minimum Derivative, Maximum Derivative, and EF
fig, axs = plt.subplots(1, 5, figsize=(25, 6), sharey=False)
sns.boxplot(data=df_min_vol, x='segmentation_type', y='MinVolume', hue='Class', ax=axs[0])
axs[0].set_title('ESVI')
axs[0].set_xlabel('')
axs[0].set_ylabel('')

sns.boxplot(data=df_max_vol, x='segmentation_type', y='MaxVolume', hue='Class', ax=axs[1])
axs[1].set_title('EDVI')
axs[1].set_xlabel('')
axs[1].set_ylabel('')
# sns.boxplot(data=df_deriv, x='segmentation_type', y='MinDeriv', hue='Class', ax=axs[2])
# axs[2].set_title('Min Derivative')
sns.boxplot(data=df_sv, x='segmentation_type', y='SV', hue='Class', ax=axs[2])
axs[2].set_title('SVI')
axs[2].set_xlabel('')
axs[2].set_ylabel('')

sns.boxplot(data=df_deriv, x='segmentation_type', y='MaxDeriv', hue='Class', ax=axs[3])
axs[3].set_title('Max Derivative')
axs[3].set_xlabel('')
axs[3].set_ylabel('')

sns.boxplot(data=df_ef, x='segmentation_type', y='EF', hue='Class', ax=axs[4])
axs[4].set_title('EF')
axs[4].set_xlabel('')
axs[4].set_ylabel('')

# Loop over all axes to rename x-tick labels based on title_dict:
for ax in axs:
    new_labels = [title_dict.get(label.get_text(), label.get_text()) for label in ax.get_xticklabels()]
    ax.set_xticklabels(new_labels)

plt.tight_layout()
plt.show()

save_filename = os.path.join(save_folder_results, 'group_comparisons.png')
fig.savefig(save_filename, dpi=300, bbox_inches='tight', pad_inches=0)


In [ ]:
# --- STEP 3: Pairwise Comparisons Between Segmentation Groups ---
# We define a helper function to perform paired tests (Wilcoxon signed-rank test)

# Compute pairwise tests for each measure:
df_pw_min_vol    = pairwise_comparisons(df_min_vol, 'MinVolume', group_col='segmentation_type', subject_col='Subject')
df_pw_max_vol    = pairwise_comparisons(df_max_vol, 'MaxVolume', group_col='segmentation_type', subject_col='Subject')
# df_pw_min_deriv  = pairwise_comparisons(df_deriv, 'MinDeriv', group_col='segmentation_type', subject_col='Subject')
df_pw_sv  = pairwise_comparisons(df_sv, 'SV', group_col='segmentation_type', subject_col='Subject')
df_pw_max_deriv  = pairwise_comparisons(df_deriv, 'MaxDeriv', group_col='segmentation_type', subject_col='Subject')
df_pw_ef         = pairwise_comparisons(df_ef, 'EF', group_col='segmentation_type', subject_col='Subject')

# Combine all pairwise results
df_pairwise = pd.concat([df_pw_min_vol, df_pw_max_vol, df_pw_sv, df_pw_max_deriv, df_pw_ef], ignore_index=True)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 24})
import seaborn as sns
sns.set_context("talk", font_scale=1.5, 
                  rc={"font.size": 18, "axes.titlesize": 18, "axes.labelsize": 18,
                      "xtick.labelsize": 18, "ytick.labelsize": 18})


# Define the order and renaming dictionary
# order = ['gt', 'pred_base', 'pred_new']
order = ['gt', 'pred']
# title_dict = {'gt':'CNN reference', 'pred_new': 'Ours', 'pred_base': 'Baseline'}

title_map = {
    'SV': 'SVI',
    'EF': 'EF',
    'MaxDeriv': 'Max. Derivative',
    'MinVolume': 'ESVI',
    'MaxVolume': 'EDVI'
}

metrics = [
    {'name': 'SV', 'data': df_sv, 'value_col': 'SV', 
     'pairwise': df_pw_sv, 'ylabel': 'Stroke Volume (SV)'},
    {'name': 'EF', 'data': df_ef, 'value_col': 'EF', 
     'pairwise': df_pw_ef, 'ylabel': 'Ejection Fraction (EF)'},
    {'name': 'MaxDeriv', 'data': df_deriv, 'value_col': 'MaxDeriv', 
     'pairwise': df_pw_max_deriv, 'ylabel': 'Max |Derivative|'},
    {'name': 'MinVolume', 'data': df_min_vol, 'value_col': 'MinVolume', 
     'pairwise': df_pw_min_vol, 'ylabel': 'Min Volume'},
    {'name': 'MaxVolume', 'data': df_max_vol, 'value_col': 'MaxVolume', 
     'pairwise': df_pw_max_vol, 'ylabel': 'Max Volume'},
]


# Assume you have your metrics and pairwise test DataFrames (e.g., df_pw_sv, etc.)
# Here, we'll show the modified annotation code within the plotting loop:
# Set up a grid with 2 rows and 3 columns (total 6 subplots), then only fill 5.
fig, axes = plt.subplots(2, 3, figsize=(18, 12), sharey=False)
axes = axes.flatten()

for ax, metric in zip(axes, metrics):
    data = metric['data']
    value_col = metric['value_col']
    pairwise_results = metric['pairwise']
    ylabel = metric['ylabel']
    metric_name = metric['name']
    
    # Create a boxplot for the metric by segmentation groups with hue for Class.
    sns.boxplot(data=data, x='segmentation_type', y=value_col, hue='Class', order=order, 
                ax=ax, palette="Set2")
    # ax.set_title(metric_name, fontsize=14)
    metric_title = title_map.get(metric_name, metric_name)
    ax.set_title(metric_title)

    # ax.set_xlabel('Segmentation', fontsize=12)
    ax.set_xlabel('')
    # ax.set_ylabel(ylabel, fontsize=12)
    ax.set_ylabel('')
    
    # --- Rename the x-ticks using title_dict ---
    # We'll replace e.g. 'gt' -> 'CNN reference', etc.
    new_labels = []
    for label in ax.get_xticklabels():
        txt = label.get_text()  # e.g. 'gt'
        new_labels.append(title_dict.get(txt, txt))  # fallback to original if not found
    ax.set_xticklabels(new_labels, rotation=0)  # rotation=0 if you want them horizontal

    # Determine the hue categories and compute the offset for the box centers.    
    # Determine the hue categories and compute the offset for the box centers.
    hue_categories = sorted(data['Class'].unique())
    n_hue = len(hue_categories)
    box_width = 0.8  # default box width in seaborn boxplots

    # For each cardiac class (hue), annotate significant differences.
    for cl in hue_categories:
        class_index = hue_categories.index(cl)
        offset = (class_index - (n_hue - 1) / 2) * (box_width / n_hue)
        
        data_cl = data[data['Class'] == cl]
        # Filter the pairwise test results for the current class.
        df_pw_cl = pairwise_results[pairwise_results['Class'] == cl]
        # Map segmentation groups to their base x-axis positions.
        x_coords = {group: order.index(group) for group in order}
        
        # Use the maximum value for the metric in the current data as baseline for annotations.
        y_max_data = data[value_col].max() * 0.9
        base_offset = y_max_data * 0.05 if y_max_data != 0 else 0.05
        num_annotations = 0
        
        # Define allowed comparisons (all lowercase, without spaces)
        allowed_comparisons = {'gtvspred_base', 'gtvspred_new', 'pred_basevspred_new'}
        
        for idx, row in df_pw_cl.iterrows():
            # Remove spaces and lowercase the comparison string.
            comp = row['Comparison'].replace(" ", "").lower()
            if comp not in allowed_comparisons:
                continue
            try:
                group1, group2 = comp.split("vs")
            except Exception:
                continue
            if group1 not in x_coords or group2 not in x_coords:
                continue
            
            # Compute x positions in data coordinates (categorical positions are 0,1,2, plus offset)
            x1 = x_coords[group1] + offset
            x2 = x_coords[group2] + offset

            if 'gt' in [group1, group2]:
                if group1 == 'gt':
                    x_symbol = x2  # put the marker over the second group
                else:
                    x_symbol = x1  # put the marker over the first group            
            else:
                # If neither is gt, place symbol at the first one
                # x_symbol = (x1 + x2) / 2.0
                x_symbol = x1
            # mid_x = (x1 + x2) / 2.0
            
            # Set y for annotation (stack annotations if multiple exist)
            y = y_max_data + base_offset * (num_annotations + 0.05)
            
            p = row['adj_p-value']
            if p < 0.05:
                # Choose symbol and color based on whether one group is ground truth.
                if 'gt' in [group1, group2]:
                    symbol = '†'      # red dagger for comparisons involving ground truth
                    symbol_color = 'red'
                else:
                    symbol = '∗'      # blue asterisk for comparisons between the two approaches
                    symbol_color = 'blue'
                # Place the symbol in data coordinates.
                ax.text(x_symbol, y, symbol, ha='center', va='bottom', 
                        color=symbol_color, fontsize=16, fontweight='bold')
                num_annotations += 1

    # Remove the default legend from each subplot
    if ax.get_legend() is not None:
        ax.get_legend().remove()

# Hide the unused subplot if you have only 5 metrics
if len(metrics) < len(axes):
    for j in range(len(metrics), len(axes)):
        axes[j].axis('off')

# Now, use the 6th subplot (axes[5]) for a combined legend.
# First, collect the handles and labels from one of the plots.
handles, labels = axes[0].get_legend_handles_labels()
# Create a vertical legend (ncol=1) that covers more of the subplot.
axes[5].legend(handles, labels, loc='center', bbox_to_anchor=(0.5, 0.5), ncol=1, 
               frameon=False, fontsize=20)
axes[5].axis('off')  # Hide the axes box for the legend subplot.
# Now, collect the handles and labels from the first subplot (or any one that had a legend).
handles, labels = axes[0].get_legend_handles_labels()

# Adjust subplot spacing so there's more room between them
fig.subplots_adjust(wspace=0.3, hspace=0.3)
plt.tight_layout()

save_filename = os.path.join(save_folder_results, 'group_comparisons_sig.png')
fig.savefig(save_filename, dpi=300, bbox_inches='tight', pad_inches=0)

save_filename = os.path.join(save_folder_results, 'group_comparisons_sig.eps')
fig.savefig(save_filename, dpi=300, bbox_inches='tight', pad_inches=0, format='eps')

save_filename = os.path.join(save_folder_results, 'group_comparisons_sig.pdf')
fig.savefig(save_filename, dpi=300, bbox_inches='tight', pad_inches=0, format='pdf')

In [ ]:
# We choose the variable to analyze (for volume, use 'Volume_Index'; otherwise, use 'Volume')
measure = 'Volume_Index'

# Compute minimum and maximum volume per subject
df_min_vol = df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])[measure].min().reset_index().rename(columns={measure: 'MinVolume'})
df_max_vol = df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])[measure].max().reset_index().rename(columns={measure: 'MaxVolume'})

# Compute EF = (Max - Min) / Max for each subject (merge min and max data)
df_ef = pd.merge(df_min_vol, df_max_vol, on=['Subject', 'segmentation_type', 'Class'])
df_ef['EF'] = (df_ef['MaxVolume'] - df_ef['MinVolume']) / df_ef['MaxVolume']
df_ef['EF'] = df_ef['EF'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Compute Stroke Volume (SV) as (MaxVolume - MinVolume)
# Merge the min and max volume dataframes
df_sv = pd.merge(df_min_vol, df_max_vol, on=['Subject', 'segmentation_type', 'Class'])
df_sv['SV'] = df_sv['MaxVolume'] - df_sv['MinVolume']

# For Derivative measures, assume df_all_merged has a 'Derivative' column.
# Compute per-subject min and max derivative for each subject, segmentation, and class.
df_deriv = (df_all_merged.groupby(['Subject', 'segmentation_type', 'Class'])['Derivative']
                .apply(lambda x: np.abs(x).max())
                .reset_index(name='MaxDeriv'))

df_ef.groupby(['segmentation_type', 'Class'])[['EF']].agg(['mean', 'std'])

In [ ]:
df_max_vol.groupby(['segmentation_type', 'Class'])[['MaxVolume']].agg(['mean', 'std'])

In [ ]:
df_min_vol.groupby(['segmentation_type', 'Class'])[['MinVolume']].agg(['mean', 'std'])

In [ ]:
df_sv.groupby(['segmentation_type', 'Class'])[['SV']].agg(['mean', 'std'])

In [ ]:
df_deriv.groupby(['segmentation_type', 'Class'])[['MaxDeriv']].agg(['mean', 'std'])